In [816]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,r2_score
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [817]:
rd = pd.read_csv('training_data.csv')
t = pd.read_csv('test_data.csv')


In [818]:
raw_data = rd.drop('AVERAGE_RAIN', axis=1)
raw_data = raw_data.drop('AVERAGE_PRECIPITATION',axis=1)

test = t.drop('AVERAGE_RAIN', axis=1)
test = test.drop('AVERAGE_PRECIPITATION',axis=1)
test = test.drop('city_name',axis=1)


raw_data['record_date'] = pd.to_datetime(raw_data.record_date, format='%Y-%m-%d %H:%M:%S')
raw_data["is_weekend"] = raw_data.record_date.dt.day_name().apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)


test['record_date'] = pd.to_datetime(test.record_date, format='%Y-%m-%d %H:%M:%S')
test["is_weekend"] = test.record_date.dt.day_name().apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)

#raw_data["record_date"] = pd.to_datetime(raw_data["record_date"]).dt.strftime("%Y%m%d").astype(float)
#test["record_date"] = pd.to_datetime(test["record_date"]).dt.strftime("%Y%m%d").astype(float)

In [819]:
# Calcular o valor mais frequente da coluna da cloudiness e substituir nos NaN
most_common = raw_data['AVERAGE_CLOUDINESS'].mode()
raw_data["AVERAGE_CLOUDINESS"].fillna(value=most_common[0] , inplace = True)

most_common1 = test['AVERAGE_CLOUDINESS'].mode()
test["AVERAGE_CLOUDINESS"].fillna(value=most_common1[0] , inplace = True)

# Quantidade de valores distintos na coluna LUMINOSITY
raw_data['LUMINOSITY'].nunique()


# Converter os valores em 'LUMINOSITY
#raw_data['LUMINOSITY']


mapper = {'DARK': 0, 'LOW_LIGHT': 1, 'LIGHT': 2}
raw_data['LUMINOSITY'] = raw_data["LUMINOSITY"].replace(mapper)

test['LUMINOSITY'] = test["LUMINOSITY"].replace(mapper)

mapper1 = {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High': 4}
raw_data['AVERAGE_SPEED_DIFF'] = raw_data["AVERAGE_SPEED_DIFF"].replace(mapper1)


In [ ]:
#con = raw_data['AVERAGE_CLOUDINESS']
#raw_data = pd.get_dummies(raw_data, columns=['AVERAGE_CLOUDINESS'])
#test = pd.get_dummies(test, columns=['AVERAGE_CLOUDINESS'])    

In [820]:
#raw_data.record_date = pd.to_datetime(raw_data.record_date)
#test.record_date = pd.to_datetime(test.record_date)

raw_data['Year'] = raw_data.record_date.dt.year.astype(int)
raw_data['Month'] = raw_data.record_date.dt.month
#raw_data['Day'] = raw_data.record_date.dt.day
raw_data['Hour'] = raw_data.record_date.dt.hour
#raw_data['Week_day'] = raw_data.record_date.dt.day_name()

test['Year'] = test.record_date.dt.year.astype(int)
test['Month'] = test.record_date.dt.month
#test['Day'] = test.record_date.dt.day
test['Hour'] = test.record_date.dt.hour
#test['Week_day'] = test.record_date.dt.day_name(locale='en')


raw_data.drop('record_date', axis=1, inplace=True)
test.drop('record_date',axis= 1, inplace=True)

#mapper2 = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3 , 'Thursday' : 4 ,'Friday': 5 , 'Saturday' : 6 , 'Sunday' : 7}
#raw_data['Week_day'] = raw_data["Week_day"].replace(mapper2)
#test['Week_day'] = test["Week_day"].replace(mapper2)


mapper3 = {'ceu limpo': 3, 'parcialmente nublado': 2, 'nublado': 1}
raw_data['AVERAGE_CLOUDINESS'] = raw_data['AVERAGE_CLOUDINESS'].replace(mapper3)
test['AVERAGE_CLOUDINESS'] = test['AVERAGE_CLOUDINESS'].replace(mapper3)






In [768]:
'''
def daypart(hour):
    if hour>=0 and hour<=7 :
        return 1
    elif hour > 7 and hour <= 11:
        return 2
    elif hour > 11 and hour <= 14:
        return 3
    elif hour > 14 and hour <= 20:
        return 2
    else : return 4
    
'''
    
#raw_data["moment_of_day"] = raw_data["Hour"].apply(daypart)


#one_hot_dates = pd.get_dummies(raw_data["Hour"])
#raw_data = pd.concat([raw_data,one_hot_dates],axis=1)






In [730]:
def season(month):
    if month in [12,1,2] :
        return 1
    elif month in [3,4,5]:
        return 2
    elif month in [6,7,8]:
        return 3
    else:
        return 4
    
raw_data["Season"] = raw_data["Month"].apply(season)

In [731]:
raw_data.drop('Month',axis=1,inplace = True)

In [808]:
raw_data

,AVERAGE_SPEED_DIFF,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,LUMINOSITY,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS,is_weekend,Year,Month,Hour
0,2,41.5,11.5,71.4,2,15.0,1019.0,100.0,3.0,2,0,2019,8,7
1,3,41.7,48.3,87.4,2,21.0,1021.0,53.0,5.0,3,0,2018,8,14
2,3,38.6,38.4,85.2,2,26.0,1014.0,61.0,4.0,2,1,2019,9,16
3,3,37.4,61.0,94.1,2,18.0,1025.0,48.0,4.0,3,0,2019,2,11
4,2,41.6,50.4,77.0,2,15.0,1008.0,82.0,10.0,2,0,2019,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6807,1,41.2,3.7,86.9,2,9.0,1030.0,71.0,4.0,3,1,2019,2,9
6808,2,38.9,51.9,76.9,2,17.0,1020.0,63.0,6.0,2,1,2019,6,18
6809,0,39.6,0.0,89.1,0,15.0,1020.0,39.0,4.0,3,0,2018,10,4
6810,0,41.6,0.0,85.5,0,8.0,1018.0,81.0,1.0,2,0,2019,1,1


In [813]:
X = raw_data.drop('AVERAGE_SPEED_DIFF',axis=1)
y = raw_data['AVERAGE_SPEED_DIFF']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2021)





In [ ]:
param_grid = {
    'max_depth': [45 ,50 ,60],
    'min_samples_leaf': [3, 4, 5,8],
    'n_estimators': [265,270,275,300]
}

grid = GridSearchCV(RandomForestClassifier(random_state=2021),param_grid,refit=True,verbose=3,cv=5)

grid.fit(X_train, y_train)


In [810]:
################### DECISION TREE #####################
#clf = DecisionTreeClassifier(max_depth=6,random_state=2021)
#clf.fit(X_train,y_train)


################# RANDOM FOREST CLASSIFIER #############
rfc=RandomForestClassifier(n_estimators = 265 , n_jobs = 2,random_state=2021)
rfc.fit(X_train,y_train)



########### GRADIENT BOOST ##############
#gradient_booster = GradientBoostingClassifier(n_estimators=45, learning_rate=0.2, max_features=4, max_depth=8, random_state=2021)
#gradient_booster.fit(X_train,y_train)


############ KNN ##########
#model = KNeighborsClassifier(n_neighbors= 6,weights = 'distance')
#model.fit(X_train,y_train)



RandomForestClassifier(n_estimators=265, n_jobs=2, random_state=2021)

In [811]:
#pred = clf.predict(X_test)
pred_rfc = rfc.predict(X_test)
#pred_gb = gradient_booster.predict(X_test)


In [812]:
accuracy_score(y_test,pred_rfc)


0.8041085840058694

In [ ]:
#print(classification_report(y_test,pred_gb))

In [814]:
test_pred = rfc.predict(test)
test_pred


array([0, 1, 0, ..., 0, 4, 1], dtype=int64)

In [815]:
res = pd.DataFrame()
res['RowId'] = range(1, len(test_pred) + 1)
res['Speed_Diff'] = test_pred

mapper2 = {0 : 'None', 1 :'Low', 2 : 'Medium', 3: 'High', 4 : 'Very_High'}
res['Speed_Diff'] = res['Speed_Diff'].replace(mapper2)
res.to_csv('submission9.csv', index=False)